## Capítulo 8. Manipulación de archivos Excel con paquetes Reader y Writer
Este capítulo introduce los paquetes:
* OpenPyXL
* XlsxWriter
* pyxlsb
* xlrd
* xlwt

Estos paquetes son usados por debajo por pandas cuando se usa la función **read_excel** y el método **.to_excel**.

Formato: Lectura -> Escritura -> Edición
xlsx: OpenPyXL -> OpenPyXL -> XlsxWriter -> OpenPyXL
xls:  xlrd -> xlwt -> xlutils

### OpenPyXL
OpenPyXL es el único paquete que permite leer y escribir archivos Excel. Además, se pueden editar archivos Excel.

#### Leyendo con OpenPyXL
Para obtener los valores de las celdas se abre el libro de trabajo Excel con **data_only=True**. El valor por defecto es False, lo cual significa que se retornan las fórmulas que están en las celdas. 

In [1]:
import pandas as pd
import openpyxl
import excel
import datetime as dt

In [3]:
#Abre el libro de trabajo Excel para leer los valores de las celdas.
#El archivo es automáticamente cerradodespués de cargar los datos.
book = openpyxl.load_workbook("xl/stores.xlsx", data_only=True)

In [4]:
book.sheetnames

['2019', '2020', '2019-2020']

In [7]:
#Se obtiene una hoja de cálculo del libro de trabajo, por nombre o índice.
sheet = book["2019"]
sheet = book.worksheets[0]

In [8]:
#Lazo para leer todas las hojas del libro.
#En vez del "name" se puede usar "title".
for i in book.worksheets:
    print(i.title)

2019
2020
2019-2020


In [9]:
#Obteniendo las dimensiones, es decir el rango usado por la hoja.
sheet.max_row, sheet.max_column

(8, 6)

In [10]:
#Lee el valor de una celda usando la notación "A1" 
# y usando los índices de las celdas, basados en 1.
sheet["B6"].value 

'Boston'

In [11]:
sheet.cell(row=6, column=2).value

'Boston'

In [4]:
#Lee en un rango de valores de celdas usando el módulo "excel".
data = excel.read(book["2019"], (2, 2), (8, 6))
data

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False],
 ['San Francisco',
  12,
  'Neriah',
  datetime.datetime(2019, 11, 2, 0, 0),
  'MISSING'],
 ['Chicago', 4, 'Katelin', datetime.datetime(2020, 1, 31, 0, 0), None],
 ['Boston', 5, 'Georgiana', datetime.datetime(2017, 4, 1, 0, 0), True],
 ['Washington DC', 3, 'Evan', None, False],
 ['Las Vegas', 11, 'Paul', datetime.datetime(2020, 1, 6, 0, 0), False]]

In [5]:
data[:2] #Imprime las primeras dos filas.

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

#### Escribiendo con OpenPyXL
OpenPyXL construye el archivo Excel en memoria y escribe el archivo una vez se llama el método **save**.
 

In [7]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
import excel

In [8]:
#Instancie un libro de trabajo Excel.
book = openpyxl.Workbook()


In [9]:
#Obtiene la primera hoja y le asigna un nombre.
sheet = book.active
sheet.title = "Sheet1"

In [10]:
#Escribe en las celdas individuales usando la notación A1 y 
# los índices de la celda, basado en 1.
sheet["A1"].value = "Hola 1"
sheet.cell(row=2, column=1, value="Hola 2")

# Formatting: fill color, alignment, border and font
font_format = Font(color="FF0000", bold=True)
thin = Side(border_style="thin", color="FF0000")
sheet["A3"].value = "Hola 3"
sheet["A3"].font = font_format
sheet["A3"].border = Border(top=thin, left=thin,
                        right=thin, bottom=thin)
sheet["A3"].alignment = Alignment(horizontal="center")
sheet["A3"].fill = PatternFill(fgColor="FFFF00", fill_type="solid")

# Number formatting (using Excel's formatting strings)
sheet["A4"].value = 3.3333
sheet["A4"].number_format = "0.00"

# Date formatting (using Excel's formatting strings)
sheet["A5"].value = dt.date(2016, 10, 13)
sheet["A5"].number_format = "mm/dd/yy"

# Formula: you must use the English name of the formula
# with commas as delimiters
sheet["A6"].value = "=SUM(A4, 2)"

# Image
sheet.add_image(Image("images/python.png"), "C1")

# Two-dimensional list (we're using our excel module)
data = [[None, "North", "South"],
        ["Last Year", 2, 5],
        ["This Year", 3, 6]]
excel.write(sheet, data, "A10")

# Chart
chart = BarChart()
chart.type = "col"
chart.title = "Sales Per Region"
chart.x_axis.title = "Regions"
chart.y_axis.title = "Sales"
chart_data = Reference(sheet, min_row=11, min_col=1,
                max_row=12, max_col=3)
chart_categories = Reference(sheet, min_row=10, min_col=2,
                        max_row=10, max_col=3)
# from_rows interprets the data in the same way
# as if you would add a chart manually in Excel
chart.add_data(chart_data, titles_from_data=True, from_rows=True)
chart.set_categories(chart_categories)
sheet.add_chart(chart, "A15")

# Saving the workbook creates the file on disk
book.save("openpyxl.xlsx")


In [11]:
#If you want to write an Excel template file, you’ll need to set the 
# template attribute to True before saving it:

book = openpyxl.Workbook()
sheet = book.active
sheet["A1"].value = "This is a template"
book.template = True
book.save("template.xltx")